In [3]:
import numpy as np
import os
import importlib
import EmbedSeg.utils.create_dicts
importlib.reload(EmbedSeg.utils.create_dicts)
from EmbedSeg.train import begin_training
from EmbedSeg.utils.create_dicts import create_dataset_dict, create_model_dict, create_loss_dict, create_configs
import torch
from matplotlib.colors import ListedColormap
import json



### Specify the path to `train`, `val` crops and the type of `center` embedding which we would like to train the network for:

The train-val images, masks and center-images will be accessed from the path specified by `data_dir` and `project-name`.
<a id='center'></a>

In [4]:
data_dir = r'C:\Users\cryst\Desktop\Organoid-Segmentation-Web-App\training'
project_name = 'Colon Organoid 5_2_2024'
project_dir = os.path.join(data_dir, project_name)
center = 'medoid' # 'centroid', 'approximate-medoid', 'medoid'

print("Project Name chosen as : {}. \nTrain-Val images-masks-center-images will be accessed from : {}".format(project_name, project_dir))

Project Name chosen as : Colon Organoid 5_2_2024. 
Train-Val images-masks-center-images will be accessed from : C:\Users\cryst\Desktop\Organoid-Segmentation-Web-App\training\Colon Organoid 5_2_2024


In [5]:
try:
    assert center in {'medoid', 'approximate-medoid', 'centroid'}
    print("Spatial Embedding Location chosen as : {}".format(center))
except AssertionError as e:
    e.args += ('Please specify center as one of : {"medoid", "approximate-medoid", "centroid"}', 42)
    raise

Spatial Embedding Location chosen as : medoid


### Obtain properties of the dataset 

Here, we read the `dataset.json` file prepared in the `01-data` notebook previously.

In [6]:
if os.path.isfile(os.path.join(project_dir, 'data_properties.json')):
    with open(os.path.join(project_dir, 'data_properties.json')) as json_file:
        data = json.load(json_file)
        one_hot, data_type, foreground_weight, n_y, n_x = data['one_hot'], data['data_type'], int(data['foreground_weight']), int(data['n_y']), int(data['n_x'])

### Specify training dataset-related parameters

Some hints: 
* The `train_size` attribute indicates the number of image-mask paired examples which the network would see in one complete epoch. Ideally this should be the number of `train` image crops. 

In the cell after this one, a `train_dataset_dict` dictionary is generated from the parameters specified here!

In [7]:
train_size = len(os.listdir(os.path.join(project_dir, 'crops', project_name, 'train', 'images')))
train_batch_size = 16 

### Create the `train_dataset_dict` dictionary  

In [8]:
train_dataset_dict = create_dataset_dict(data_dir = os.path.join(project_dir, 'crops'),
                                         project_name = project_name,  
                                         center = center, 
                                         size = train_size, 
                                         batch_size = train_batch_size, 
                                         one_hot = one_hot,
                                         type = 'train')

`train_dataset_dict` dictionary successfully created                 with: 
 -- train images accessed from C:\Users\cryst\Desktop\Organoid-Segmentation-Web-App\training\Colon Organoid 5_2_2024\crops\Colon Organoid 5_2_2024\train\images, 
 -- number of images per epoch equal to 821, 
 -- batch size set at 16, 


### Specify validation dataset-related parameters

Some hints:
* The size attribute indicates the number of image-mask paired examples which the network would see in one complete epoch. Here, it is recommended to set `val_size` equal to the total number of validation image crops. For example, for the `dsb-2018` dataset, we notice ~2600 validation crops.

In the cell after this one, a `val_dataset_dict` dictionary is generated from the parameters specified here!



In [9]:
val_size = len(os.listdir(os.path.join(project_dir, 'crops', project_name, 'val', 'images')))
val_batch_size = 16

### Create the `val_dataset_dict` dictionary

In [10]:
val_dataset_dict = create_dataset_dict(data_dir = os.path.join(project_dir, 'crops'),
                                       project_name = project_name, 
                                       center = center, 
                                       size = val_size, 
                                       batch_size = val_batch_size, 
                                       one_hot = one_hot,
                                       type ='val',)

`val_dataset_dict` dictionary successfully created                 with: 
 -- val images accessed from C:\Users\cryst\Desktop\Organoid-Segmentation-Web-App\training\Colon Organoid 5_2_2024\crops\Colon Organoid 5_2_2024\val\images, 
 -- number of images per epoch equal to 87, 
 -- batch size set at 16, 


### Specify model-related parameters

Some hints:
* Set the `input_channels` attribute equal to the number of channels in the input images. 

In the cell after this one, a `model_dataset_dict` dictionary is generated from the parameters specified here!

In [11]:
input_channels = 1

### Create the `model_dict` dictionary

In [12]:
model_dict = create_model_dict(input_channels = input_channels)

`model_dict` dictionary successfully created                 with: 
 -- num of classes equal to 1, 
 -- input channels                 equal to [4, 1], 
 -- name equal to branched_erfnet


### Create the `loss_dict` dictionary

In [13]:
loss_dict = create_loss_dict(foreground_weight=2, w_seed=5)

`loss_dict` dictionary successfully created                 with: 
 -- foreground weight equal to 2.000, 
 -- w_inst                 equal to 1, 
 -- w_var                 equal to 10, 
 -- w_seed equal to 5


### Specify additional parameters 

Some hints:
* The `n_epochs` attribute determines how long the training should proceed. In general for reasonable results, you should atleast train for longer than 50 epochs.
* The `save_dir` attribute identifies the location where the checkpoints and loss curve details are saved. 
* If one wishes to **resume training** from a previous checkpoint, they could point `resume_path` attribute appropriately. For example, one could set `resume_path = './experiment/dsb-2018-demo/checkpoint.pth'` to resume training from the last checkpoint. 

In the cell after this one, a `configs` dictionary is generated from the parameters specified here!
<a id='resume'></a>

In [14]:
n_epochs = 200
save_dir = os.path.join(project_dir, "model")
resume_path  = None

### Create the  `configs` dictionary 

In [15]:
configs = create_configs(n_epochs = n_epochs,
                         one_hot = one_hot,
                         resume_path = resume_path, 
                         save_dir = save_dir, 
                         n_y = n_y, 
                         n_x = n_x,)

`configs` dictionary successfully created with: 
 -- n_epochs equal to 200, 
 -- save_dir equal to C:\Users\cryst\Desktop\Organoid-Segmentation-Web-App\training\Colon Organoid 5_2_2024\model, 
 -- n_z equal to None, 
 -- n_y equal to 1392, 
 -- n_x equal to 1392, 


### Begin training!

Executing the next cell would begin the training. 

In [16]:
begin_training(train_dataset_dict, val_dataset_dict, model_dict, loss_dict, configs)

2-D `train` dataloader created! Accessing data from C:\Users\cryst\Desktop\Organoid-Segmentation-Web-App\training\Colon Organoid 5_2_2024\crops\Colon Organoid 5_2_2024/train/
test
Number of images in `train` directory is 821
Number of instances in `train` directory is 821
Number of center images in `train` directory is 821
*************************
2-D `val` dataloader created! Accessing data from C:\Users\cryst\Desktop\Organoid-Segmentation-Web-App\training\Colon Organoid 5_2_2024\crops\Colon Organoid 5_2_2024/val/
test
Number of images in `val` directory is 87
Number of instances in `val` directory is 87
Number of center images in `val` directory is 87
*************************
Creating Branched Erfnet with [4, 1] outputs
Initialize last layer with size:  torch.Size([16, 4, 2, 2])
*************************
Created spatial emb loss function with:                     n_sigma: 2, foreground_weight: 2
*************************
Created logger with keys:  ('train', 'val', 'iou')
Starting e

100%|██████████| 6/6 [00:05<00:00,  1.08it/s]


===> train loss: 2.16
===> val loss: 1.91, val iou: 0.46
=> saving checkpoint
Starting epoch 1
learning rate: 0.0004977494364660346


100%|██████████| 6/6 [00:04<00:00,  1.26it/s]


===> train loss: 1.54
===> val loss: 1.38, val iou: 0.57
=> saving checkpoint
Starting epoch 2
learning rate: 0.0004954977417064171


...

===> train loss: 0.25
===> val loss: 0.72, val iou: 0.69
=> saving checkpoint
Starting epoch 198
learning rate: 7.924465962305572e-06


100%|██████████| 6/6 [00:04<00:00,  1.21it/s]


===> train loss: 0.26
===> val loss: 0.72, val iou: 0.67
=> saving checkpoint
Starting epoch 199
learning rate: 4.2466161615856206e-06


100%|██████████| 6/6 [00:04<00:00,  1.24it/s]

===> train loss: 0.25
===> val loss: 0.72, val iou: 0.68
=> saving checkpoint


<div class="alert alert-block alert-warning"> 
  Common causes for errors during training, may include : <br>
    1. Not having <b>center images</b> for  <b>both</b> train and val directories  <br>
    2. <b>Mismatch</b> between type of center-images saved in <b>01-data.ipynb</b> and the type of center chosen in this notebook (see the <b><a href="#center"> center</a></b> parameter in the third code cell in this notebook)   <br>
    3. In case of resuming training from a previous checkpoint, please ensure that the model weights are read from the correct directory, using the <b><a href="#resume"> resume_path</a></b> parameter. Additionally, please ensure that the <b>save_dir</b> parameter for saving the model weights points to a relevant directory. 
</div>